In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from collections import Counter

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seojin/data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seojin/data/test.csv')

In [ ]:
# train_df n개 이하 제거

In [ ]:
train_df.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."


In [ ]:
def filter_keywords_by_class_count(df, min_count):
    """분류별로 n회 이하로 등장한 키워드를 제거한 DataFrame과 삭제된 키워드를 반환합니다."""
    # 모든 분류에 대해 키워드 집계
    class_keyword_counts = {}

    for label in df['분류'].unique():
        class_df = df[df['분류'] == label]
        keywords = class_df['키워드'].str.cat(sep=',').split(',')
        counts = Counter(keywords)
        class_keyword_counts[label] = counts

    # min_count 이하인 키워드 찾기
    low_count_keywords = {}
    for label, counts in class_keyword_counts.items():
        low_count_keywords[label] = {keyword for keyword, count in counts.items() if count <= min_count}

    # DataFrame의 키워드에서 각 분류별로 low_count_keywords를 제거
    def filter_keywords(row):
        keywords = row['키워드'].split(',')
        filtered_keywords = [keyword for keyword in keywords if keyword not in low_count_keywords[row['분류']]]
        return ','.join(filtered_keywords)

    # 새로운 열에 필터링된 키워드 저장
    df['필터된_키워드'] = df.apply(filter_keywords, axis=1)
    return df, low_count_keywords


In [ ]:
train_df, removed_keywords = filter_keywords_by_class_count(train_df, min_count=3)

In [ ]:
train_df.head()

,ID,분류,제목,키워드,필터된_키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트...","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화...","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특...","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코...","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,대사..."


In [ ]:
지역_value = removed_keywords.get('문화:전시_공연')
print(지역_value)

In [ ]:
# # 키워드와 필터된 키워드를 각각 리스트로 변환 후 차이를 구합니다.
# original_keywords = set(train_df.키워드[3].split(','))
# filtered_keywords = set(train_df.필터된_키워드[3].split(','))

# # 원래 있던 키워드에서 필터된 후 사라진 키워드 확인 (차집합)
# removed_keywords = original_keywords - filtered_keywords

# print("원래 키워드:", original_keywords)
# print("필터된 키워드:", filtered_keywords)
# print("제거된 키워드:", removed_keywords)

원래 키워드: {'지위', '지방행정환경', '유지', '2018년', '예산', '용인시민', '강화', '백군기', '환영사', '경기용', '수요', '사무', '염태영', '행정수요', '변화', '가치', '개발제한구역', '확보', '도시', '상대적', '실질적', '전시장', '진전', '32년', '영광', '시장', '분권법', '광역', '시민', '서비스', '재정적', '평가', '재정분권', '진입', '맞춤', '통과', '재정력', '광역지자체', '행정', '조성', '자족도시', '출범', '합리적', '지방', '플랫폼시티', '도시경쟁력', '특례시추진공동대응기구', '고양', '국가', '제공', '부여', '핵심', '경남', '일산테크노밸리', '형태', '전략적', '삼중규제', '추진', '경기', '대비', '창원시', '고양군', '6년', '인구', '대도시', '중간', '특례', '광역시', '공무원', '특례조항', '가능', '속도', '제도화', '차별화', '지정', '기초지자체', '지원', '제약', '국회', '과밀억제권역', '명칭', '이양', '중앙', '국가균형발전', '격상', '특례시', '반도체', '100만', '차별', '광역자치단체', '단체', '용인특례시', '기초', '건설', '공약', '주인', '기초지방정부', 'CJ', '숙원', '브랜드', '개정안', '사무이양', '용인시', '군사보호구역', '개정', '자치', '출발점', '바탕', '규모', '광역권한', '킨텍스', '지속적', '이재준', '본회의', '일반', '노력', '우리나라', '초석', '수원', '도내', '권한', '위상', '생각', '재정', '수원시', '유형', '123만', '라이브시티', '민선', '확대', '방송영상밸리', '골자', '자랑', '용인', '승격', '단계', '실현', '대응', '클러스터', '지방소멸위기', '부탁', '미래', '자치분권', '고양시

In [ ]:
# test_df n 개 이하제거

In [7]:
removed_keywords.keys()

NameError: name 'removed_keywords' is not defined

In [ ]:
# 제거된 키워드를 합쳐서 test_df에서 필터링
all_removed_keywords = set.union(*removed_keywords.values())

In [ ]:
def filter_test_keywords(test_df, removed_keywords):
    """test_df에서 제거된 키워드를 삭제한 DataFrame을 반환합니다."""
    def filter_keywords(row):
        keywords = row['키워드'].split(',')
        filtered_keywords = [keyword for keyword in keywords if keyword not in removed_keywords]
        return ','.join(filtered_keywords)

    # 필터링된 키워드로 업데이트
    test_df['필터된_키워드'] = test_df.apply(filter_keywords, axis=1)
    return test_df


In [ ]:
# test_df에서 키워드 필터링
test_df = filter_test_keywords(test_df, all_removed_keywords)


In [ ]:
test_df.head()

,ID,제목,키워드,필터된_키워드
0,TEST_00000,[부고] 김태수씨 별세 외,"김태수,별세,김태수씨,서울,광남초등학,교장,별세,김윤정,이노코리아,대표,희정,한성대...","광남초등학,도식씨,경기,용인시"
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력","신규,확진,나흘,세자릿수,방역당국,핼러윈,풍선,효과,차단,총력,감염증,신종,코로나바...","경기,지인모임,경기,용인시,경기,행복해요,경기,SRC재활병원,경기,경기,경기,992건"
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...","전해철,장관,재정,분권,강화,지방자치,2.0,시대,마중물,마련,장관,전해철,행정안전...","시작,당정청,재정분권특별위원회,경기,용인시,특례시지원협의회,관광개발조합,특별자지단체..."
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사","용인시,구인,장애인,구직,만남,채용,행사,노호근,용인특례시,장애인,취업,지원,대회의...","용인시,용인시,용인시"
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,"지자체,경기,북동부,지역,산업단지,혁신단위,설정,전략,지역,연계,특성,제시,경기도경...","경기,경기,연구작업,북동부지역,고양테크노밸리,고양테크노밸리,IP융ㆍ복합클러스터,섬유..."


In [ ]:
# train_df와 test_df에서 'ID', '제목', '키워드' 열 삭제
train_df = train_df.drop(columns=['ID', '제목', '키워드'])
test_df = test_df.drop(columns=['ID', '제목', '키워드'])

# 결과 확인
print("train_df 열 삭제 후:\n", train_df.head())
print("test_df 열 삭제 후:\n", test_df.head())


train_df 열 삭제 후:
          분류                                            필터된_키워드
0  문화:전시_공연  용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화...
1        지역  용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트...
2        지역  용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화...
3        지역  수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특...
4        국제  용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,대사...
test_df 열 삭제 후:
                                              필터된_키워드
0                                   광남초등학,도식씨,경기,용인시
1    경기,지인모임,경기,용인시,경기,행복해요,경기,SRC재활병원,경기,경기,경기,992건
2  시작,당정청,재정분권특별위원회,경기,용인시,특례시지원협의회,관광개발조합,특별자지단체...
3                                        용인시,용인시,용인시
4  경기,경기,연구작업,북동부지역,고양테크노밸리,고양테크노밸리,IP융ㆍ복합클러스터,섬유...


In [ ]:
train_df = train_df.rename(columns={'필터된_키워드': '키워드'})

train_df.head()

,분류,키워드
0,문화:전시_공연,"용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,지역,"용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,지역,"용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,지역,"수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,국제,"용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,대사..."


In [ ]:
test_df = test_df.rename(columns={'필터된_키워드': '키워드'})

test_df.head()

,키워드
0,"광남초등학,도식씨,경기,용인시"
1,"경기,지인모임,경기,용인시,경기,행복해요,경기,SRC재활병원,경기,경기,경기,992건"
2,"시작,당정청,재정분권특별위원회,경기,용인시,특례시지원협의회,관광개발조합,특별자지단체..."
3,"용인시,용인시,용인시"
4,"경기,경기,연구작업,북동부지역,고양테크노밸리,고양테크노밸리,IP융ㆍ복합클러스터,섬유..."


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

In [ ]:
config = {
    "learning_rate": 2e-5,
    "epoch": 3,
    "batch_size": 64
}

CFG = SimpleNamespace(**config)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# 데이터 준비
#train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
#test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [ ]:
train_dataset[0]

{'text': '매물,거래량,조짐,집값,안정,아시아투데이,서울,포함,매물,수도,아파트,거래량,매수세,추세,주택,시장,안정,주목,부동산,시장,관계자,전문가들,중과,보유세,세제,인상,매물,적체,현상,공급대책,관망세,영향,정부,인상,부동산,세제,본격적,시행,부담,다주택자들,매물,시작,역대급,물량,2,사전,청약,예고,무주택자들,매수,분위기,해석,아실,부동산,빅데이터,업체,아파트실거래가,매물,이날,기준,서울,아파트,매매,전세,월세,지난달,11.3%,7만,8만,경기,9만,이달,11.7%,서울,은평구,이달,24.2%,매물,강남3구,서초구,22.2%,송파구,10.1%,강남구,1만,1만,강남구,자치구,서울,매물,상태,경기도,안산시,단원구,38.7%,의왕시,30.5%,973건,의정부시,27.2%,강남,시세,분당구,경기,성남시,25.4%,수지구,경기,용인시,경기,하남시,매물,거래량,집계,이날,12월,3월,18일,기준,거래량,절반,추세,서울,거래량,1월,급감,3월,중순,이날,거래량,거래량,비교,10분,수준,실거래가,하락,추세,부동산114,강남,은마아파트,거래,이달,23억,거래,송파,헬리오시티,17억,거래,17억,2000만,매물,상태,관계자,서초구,매물,거래,보유세,양도세,부담,다주택자들,연초,사람,매매가,분위기,본부장,장재현,리얼투데이,6월,보유세,부담,매물,2,영향,영향,다주택자들,부담,토지,부소장,거래량,감소,추세,매매가,소득,괴리,30대,구매,소진,시장,상황,보유세,인상,2,복합적,작용,결과,부소장,매매거래량,서울,아파트,매매,거래량,6월,7월,제외,비정상적,가격,모습,전형적,대세,하락,초입,현상,거래량,급매물,본격적,시작,설명',
 'input_ids': tensor([   2,    0,   46,    0,   46,    0,   46,    0,   46, 6817,   46, 6802,
           46, 6556,   46,    0,   46,    0,   46,    0,   46, 6809,   46,    0,
           46,    0

In [ ]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 학습
model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 171/171 [00:43<00:00,  3.96it/s]


Validation F1 Score: 0.17


Validating: 100%|██████████| 171/171 [00:42<00:00,  3.98it/s]


Validation F1 Score: 0.29


Validating: 100%|██████████| 171/171 [00:43<00:00,  3.93it/s]

Validation F1 Score: 0.37


In [ ]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [00:42<00:00,  8.59it/s]


In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seojin/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/seojin/baseline_kobert.csv", encoding='UTF-8-sig', index=False)

In [5]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seojin/baseline_kobert.csv")
sample_submission.head()

,ID,분류
0,TEST_00000,스포츠:올림픽_아시안게임
1,TEST_00001,스포츠:축구
2,TEST_00002,스포츠:올림픽_아시안게임
3,TEST_00003,사회:장애인
4,TEST_00004,스포츠:올림픽_아시안게임


In [6]:
sample_submission['분류'].value_counts()

,count
분류,
지역,11005
스포츠:올림픽_아시안게임,9007
사회:장애인,2246
정치:국회_정당,684
스포츠:축구,220
문화:출판,149
문화:전시_공연,54
사회:사회일반,12
IT_과학:모바일,6
